In [1]:
include("src/JuTrack.jl")

Main.JuTrack

In [2]:
using .JuTrack
using Enzyme
using Test
using BenchmarkTools
include("test/ssrf_ring.jl")

ssrf (generic function with 1 method)

In [21]:
#CTPS = CTPS (intial value, what palce in array that derv occupies, # of unique variables finding derv of, order of calculation)

In [22]:
function fodo_ring(K1, RAD)
        D1 = DRIFT(name="D1", len=0.34)
        D2 = DRIFT(name="D2", len=0.59)
        QF1 = KQUAD(name="QF1", len=0.32, k1=K1, rad=RAD )
        QD1 = KQUAD(name="QD1", len=0.32, k1=-1.192160, rad=RAD  )
        BendingAngle = pi/2
        BD1 = SBEND(name="BD1", len=0.72, angle=BendingAngle/2, e1=BendingAngle/2, e2=0.0 , rad=RAD )
        BD2 = SBEND(name="BD2", len=0.72, angle=BendingAngle/2, e1=0.0, e2=BendingAngle/2, rad=RAD  )

        FODO = (QF1, D1, QD1, D1, QF1)
        BD1_comp = (BD1)
        BD2_comp = (BD2)
        D2_comp = (D2)

        CELL = Vector{AbstractElement}(undef, 8)
        for i in eachindex(FODO)
                CELL[i] = FODO[i]
        end
        CELL[length(FODO)+1] = BD1
        CELL[length(FODO)+2] = D2
        CELL[length(FODO)+3] = BD2
                # CELL[i+172] = M5[i]
        
        ELIST = Vector{AbstractElement}(undef, 4*length(CELL))
        for i in eachindex(CELL)
                ELIST[i] = CELL[i]
                ELIST[i+length(CELL)] = CELL[i]
                ELIST[i+2*length(CELL)] = CELL[i]
                ELIST[i+3*length(CELL)] = CELL[i]
        end
        return ELIST
end

function track(k1_in)
        # particles = [0.001 0.0001 0.0005 0.0002 0.0 0.0; 0.001 0.0 0.0 0.0 0.0 0.0] #There is 2 particles here, and their 6 vector
        particles = [.001 .0001 .0005 .0002 0.0 0.0] 
        beam = Beam(particles)
        line = fodo_ring(k1_in[1],0.0)
        linepass!(line, beam)
        return beam.r
end

function track_TPS(k1_in)
        line = fodo_ring(k1_in[1],0.0)
        x = CTPS(0.001, 1, 6, 3)
        xp = CTPS(0.0001, 2, 6, 3)
        y = CTPS(0.0005, 3, 6, 3)
        yp = CTPS(0.0002, 4, 6, 3)
        delta = CTPS(0.0, 5, 6, 3)
        z = CTPS(0.0, 6, 6, 3)
        rin = [x, xp, y, yp, delta, z]
        linepass_TPSA!(line, rin)
        return rin[1].map
end

#Between track() and track_TPS(), it seems I am only able to have one particle through TPS? Create function to loop through multiple? 

function twiss(k1_in)
        fodo = fodo_ring(k1_in[1],0)
        twiss_in = EdwardsTengTwiss(betax=2.0, betay=3.0)
        twiss_out = Twissline(twiss_in, fodo, 0.0, 1, length(fodo) )[3] #(Twiss In, ring/line, dp, order, end index)
        return twiss_out
end

twiss (generic function with 1 method)

In [23]:
#Want to compare the output of track() and track_TPS() to see if they are the same
#Will compare the second particle in the beam, starting at [.001 .0001 .0005 .0002 0.0 0.0]

test_k1 = [-1.063770]
println(track(test_k1))
println(track_TPS(test_k1))
#Shouldn't these be the same first value? --Resolved by fixing bending angle from pi/20 -> pi/2

grad1 = gradient(Forward, track_TPS, test_k1)
grad2 = autodiff(Forward, track, DuplicatedNoNeed, Duplicated(test_k1[1], 1.0))[1]  #gives first derv of r w.r.t. k1

println(grad1)
println(grad2)
#Shouldn't these be the same first value? --Resolved by fixing bending angle from pi/20 -> pi/2



[0.7112164003616027 0.2872815446630527 0.001446276155597898 -0.0015768918422392098 1.4539669177801027 0.0]
[0.7112164003616027, 631.4075529160766, 798.0884744552595, 0.0, 0.0, 0.0, 515.4103797881224, 0.0, 0.0, 0.0, 0.0, 0.0, -533.7570083934322, 0.0, 0.0, 0.0, 0.0, -1046.8938807332509, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -671.7742907194936, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -1000.8283723455415, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -653.6506223545894, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -43.7583250105088]
([-1.4801082967383716, -1341.9364632979361, -1381.7183344043742, 0.0, 0.0, 0.0, -636.719145416376, 0.0, 0.0, 0.0, 0.0, 0.0, 1999.371702668934, 0.0, 0.0, 0.0, 0.0, 2825.166006404621, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1285.9827081016233, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.

In [3]:
# x = CTPS(3.0, 1, 6, 3)
# y = CTPS(2.0, 2, 6, 3)
# println(y.map[1])

# track(-1)
# track_TPS(-1)

# println(twiss([-1.063]))

In [18]:

# println(track_TPS(-1.063770))
# println(length(track_TPS(-1.063770)))
# x = [-1.063770]
# println(track_TPS(x))
# @btime track_TPS(x) 
# @btime grad = gradient(Forward, test_track_TPS, x)

In [25]:

# println(track(-1.063770))
# x = [-1.063770]
# println(track(x))
# @time track(x) 
# @time grad = gradient(Forward, track, x)
# println(gradient(Forward, track, x))

  0.000087 seconds (417 allocations: 49.297 KiB)
  0.000626 seconds (1.77 k allocations: 101.719 KiB)


([-1.4801082967383705 -0.6556533110612008 … -3.0926848939997442 0.0],)